In [10]:
import pandas as pd
import sqlite3

• create a connection to the database using the library sqlite3

In [11]:
db_con = sqlite3.connect('../checking-logs.sqlite.sqlite')

<h2>• using only one query for each of the groups, create two dataframes: test_results
and control_results with the columns time and avg_diff and only two rows</h2><br>
◦ time should have the values: after and before<br>
◦ avg_diff contains the average delta among all the users for the time period
before each of them made their first visit to the page and afterward<br>
◦ only take into account the users that have observations before and after<br>
<h2>• we still are not using the lab ’project1’</h2><br>

In [12]:
query = '''
--count average
SELECT case when test.first_commit_ts < test.first_view_ts
        THEN 'before' else 'after'
        end as time,
        AVG(CAST((julianday(test.first_commit_ts) -
      	julianday(datetime(deadlines.deadlines, 'unixepoch')))
     	* 24 AS integer)) AS avg_diff
FROM test
JOIN deadlines ON test.labname=deadlines.labs
WHERE labname<>'project1'
--users with after AND before values
AND uid in 
(SELECT uid from
 --before,after commit
(SELECT test.uid,
        case when test.first_commit_ts < test.first_view_ts
        THEN 'before' else 'after'
        end as time
FROM test
JOIN deadlines ON test.labname=deadlines.labs
WHERE labname<>'project1')
GROUP by uid
HAVING COUNT(DISTINCT time)=2)
GROUP by time
'''
test_results = pd.io.sql.read_sql(query, db_con)
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [13]:
query = '''
--count average
SELECT case when control.first_commit_ts < control.first_view_ts
        THEN 'before' else 'after'
        end as time,
        AVG(CAST((julianday(control.first_commit_ts) -
      	julianday(datetime(deadlines.deadlines, 'unixepoch')))
     	* 24 AS integer)) AS avg_diff
FROM control
JOIN deadlines ON control.labname=deadlines.labs
WHERE labname<>'project1'
--users with after AND before values
AND uid in 
(SELECT uid from
 --before,after commit
(SELECT control.uid,
        case when control.first_commit_ts < control.first_view_ts
        THEN 'before' else 'after'
        end as time
FROM control
JOIN deadlines ON control.labname=deadlines.labs
WHERE labname<>'project1')
GROUP by uid
HAVING COUNT(DISTINCT time)=2)
GROUP by time
'''
control_results = pd.io.sql.read_sql(query, db_con)
control_results

,time,avg_diff
0,after,-117.636364
1,before,-99.464286


<h2>have the answer: did the hypothesis turn out to be true and the page does affect
the students’ behavior?</h2><br>
Answer: hypothesis is wrong